In [49]:
import requests
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
path = 'C:/Users/82108/Desktop/chromedriver.exe'

driver = webdriver.Chrome(path) # chromedriver 열기


def main():
    global driver, load_wb, review_num, searchList,place_name
    
    searchList = []

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/?')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['왕십리역 카페']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)
        
    driver.quit()   
    print(searchList)    
    print("finish")
    

def search(place):
    global driver, place_name

    search_area = driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]')  # 검색 창
    
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem')  # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    print(place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath(
            '//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        while True:
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem')  # 장소 목록 list

                print(place_lists)

                crawling(place, place_lists)
            
            xPath = '//*[@id="info.search.page.next"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        if i >= 3:
            i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        
        for i, review in enumerate(review_lists):
            
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                innersearchList = []
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(rating[0].text)
                else:
                    val = comment[0].text + '/0'
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(0)
                print(val)
                
                searchList.append(innersearchList)

    else:
        print('no review in extract')
        ret = False
    return ret
    
    
if __name__ == "__main__":
    main()

##### 0
#### 인더매스 마장
no review in extract
아메리카노 종류 3 개 중에서 고를 수 있어요.  드립도 있고  직원분이 처음에 갔을 때는 불친철했는데 오늘은 친절한 분이셨네요~  빵 종류는 적당히 있어요  케이크 종류는 적음 /5
커피 맛있고 분위기 좋아서 자주 가는데   실내에 모기가 진짜 많아요 ㅠㅠ/3
딱히 특별하진 않았음 /3
커피 맛있어요/5
레몬 파운드케이크 너무 맛있어서 놀랏음 커피도 굿굿굿 솔직히 집 앞이였음 일주일에 세번정도는 파운드케이크 사먹으러 갔을 듯 /5
최고예요/5
최고예요/5
커피 맛있어요 레몬파운드케익도 최고/5
음악, 인테리어, 커피, 디저트류 모두 대만족/5
밖에서 차려입고  한참 사진 찍길래 기대하고 들어갔는데 너무 비싸요 디저트도 쏘쏘/3
커피 맛있어요  구석 자리 앉았는데 멍 때리면서 여유 부릴수 있어서 좋았습니다 뜬금 없는 곳에 카페가 있는 느낌이긴한데 인기가 많은 곳임을 알수 있어요 /5
인스타 핫플 되기 전부터 방문했었어요 집 앞 공장?같던곳이 갑자기 조명달고 그러길래 뭐지 했는데 카페였어서 놀랐던 기억... 라떼류 진짜 너무 맛있어요 친구들 추천할때마다 맛있다고 인정함...이사해서 자주 못가는게 너무 아쉬움/5
최고/5
매장넓어서 좋았고 밀크티 맛있어요~/4
앉은 자리에 콘센트가 망가졌는지 잘 안되서 그거 하나 불편했어요! 매장 넓고 좋아요/4
애플시나몬라떼 좀 달지만 피로를 달래는 맛/4
별로예요/1
최고예요/5
와이파이 느리고 콘센트 전원 내려가기도 해서, PC 사용하기에는 안좋아요~/2
창고형 카페. 맛 좋음. 치즈케익 특이함./4
최고예요/5
음료도 케익도 훌륭하지만 한물간 인스타핫플이라는게 마음이 좀 짠해요/3
Coffee/4
커피는 여기가 최고 분위기 넓고, 인테리어심플 /4
반려동물 입장 가능해서 좋아요 대신 꼬옥 안고 있어야해요 쾌적하고 넓어서 탁 트이는 기분ʺ̤ /5
집근처인데 자주 애용하려구요 조용하고 인테리어 화이트톤 심플해서 맘이 편해지는 곳이에요 음료들 퀄리티도 완

no review in crawling
no review in crawling
#### 아리가또
커피 맛있다는 후기 보고 간거였는데  저랑 남자친구는 커피 맛은 별로였어요...  하지만 제리치즈케이크는 맛있었어요!!!!!!!! 그리고 현금으로 결제하면 아메리카노랑 라떼는 사이즈업 되더라구요!  /3
음료 케이크 다 맛있음. 특히 카페라테가 맛있음. 스텐 텀블러나 컵에 담아 먹으면 더 맛있음. /5
3시간있으면 칼같이 쫓아냄/1
조금 아쉬워요/2
좋아요/4
티라미수 맛있음/5
공부하는사람들이 좀 많고 가토쇼콜라 맛있음    /4
좋아요/4
최애가페/5
좋아요/4
왕십리 최고의 카페 공부하기 좋고 커피맛도 좋고!/5
좋아요/4
좋아요/4
일호점은 왕십리 바로 옆이라, 약속장소로 괜찮아요.  그리고 아메리카노나 라떼도 맛있음ㅎ/4
최고예요/5
여기 좋아요. 분위기 좋고, 직원들 왕친절!/5
no review in crawling
no review in crawling
#### 팀메리
그릭요거트 찾아다니면서 먹구 만들어서 먹는 인간인데여.. 들리는 것만큼 극찬하거나 특별한 맛은 아늼다.. /3
최고예요/5
요거트 진짜 맛있어요ㅠㅠㅠㅠㅠ/5
여기서 일주일에 삼만원은 쓰는 것 같음 그정도로 맛있음/5
진짜 요거트 먹었다고 느낄 수 있어요! /5
완전 맛있어요♡ 친절 그자체♡/5
최고예요/5
양에 비해 가격이 좀 비싼 편이지만 맛있어요. 요거트가 꾸덕하고 신선해요. 주문할 때 메뉴마다 뭐 들어가는지 헷갈리긴 해요. 포장할 때 먹을때까지 텀이 좀 있으면 그래놀라가 불고 눅눅해져서 그건 따로 포장하시는 편이 좋을 것 같아요/4
레이첼럽이 젤맛있어효 팀메리는 시그니처였는데 저는그냥 그랬어요/5
매일 가고 싶은 팀메리 (´ ▽｀).。ｏ♡/5
매주 요거트 500g 하나랑 그래놀라 사가요 맛있기도 하고, 아침식사 대용으로 먹기 너무 좋아요 ㅎㅎ 동네에 이런 좋은 곳이 생겨서 너무 좋아요! 사장님도 항상친절하시구 챙겨주세요!  정말 추천합니당 :) /5
맛있어서 쌓아두고 먹고싶어요 하

[<li class="PlaceItem clickArea">
<div class="utile_item" data-id="btnsWrap">
<span class="screen_out" data-id="screenOutName">플레이그라운드</span>
<a class="fav" data-id="fav" href="#none">
<span class="ico_fav">즐겨찾기</span><span class="num"></span>
</a>
<a class="roadview" data-id="roadview" href="#none">로드뷰</a>
<button class="DirectionItemInfoBtn" data-id="routeBtn" type="button">길찾기</button>
<div class="FlagItemInfoLayer HIDDEN">
<div class="origin" data-id="origin">출발</div>
<div class="via" data-id="via">경유</div>
<div class="dest" data-id="dest">도착</div>
</div></div>
<div class="thumb_item HIDDEN">
<img alt="" class="thumb_g HIDDEN" data-id="image" height="70" src="" width="70"/>
</div>
<div class="head_item clickArea">
<strong class="tit_name">
<a class="link_order clickArea" data-id="icon" href="#none">A</a>
<a class="link_name" data-id="name" href="#none" title="플레이그라운드">플레이그라운드</a>
</strong>
<span class="subcategory clickable" data-id="subcategory">카페</span>
</div>
<div class="rating

#### 플레이그라운드
no review in extract
깜장일 때도 가끔 왔었는데, 바뀌고 나서도 공부하기는 더할 나위없이 좋은 분위기 굿이다/5
좋아요/4
왕십리에 이런 분위기 좋은 카페가 생기다닝 커피도 와방 맛있어요:D/5
no review in crawling
no review in crawling
#### 메가커피 왕십리점
no review in extract
no review in crawling
no review in crawling
#### 마음온도카페
맛있습니다 또 취지가 상당히 좋은 기업이라 마음에 듭니다/5
분위기도 맛도 정말 좋아요ㅠㅠ 느끼한 거 싫어하시는분들은 매운 까르보나라 드셔보세요~ 추천합니당!:D/5
분위기 좋고, 편하게 있기 좋았어요,  친구랑 고르곤졸라 피자하고, 매운 까르보나라 먹었는데 맛있었습니다. ^-^/5
분위기 개좋음../5
피자를 주문해서 첫 조각을 먹는데 벌레가 나왔어요. 야외에서 피자를 굽는다고, 그래서 벌레가 들어간 것 같다고 하시는데, 그럼 야외에서 조리를 하시면 더욱 신경을 써야 하는게 아닐까요???? /1
no review in crawling
no review in crawling
#### 할리스커피 한양대시티빌점
영업중이라매/1
대학가라서 그런건 알겠지만 카페에서 공부하는 걸로 평가하는 건 좀 그럼. 나쁘지 않게 넓고 대화 나누기 괜찮은 카페./4
테이블이 너무 다닥다닥 붙어있음 /2
좋습니다/5
오픈시간이 늦습니다/1
스벅보다 덜 붐비니 좋네요 커피 맛도 많이 좋아졌고/4
별 한개도 아깝네요/1
no review in crawling
no review in crawling
#### 설빙 한양대점
no review in extract
메론빙수 먹다가 메론을 포크로 먹기 힘들어서 스푼하나 달라고했더니 안주시네요....쩝..../1
설빙은 맛있죠~ 근데 메론빙수 먹는데 포크하나 더 줄수 있냐는데 안된다네요ㅠㅠ 그리고 이 매장이 유독 음악소리사 크고 시끄러워요..;/2
별로예요/1
주문누락 딸기냉동 청

#### 놀숲 한양대점
no review in extract
추억돋는 만화방/3
좋아요/4
쾌적하고 라면도 맛있고! 시간가는 줄 모르고 놀았네요~!/4
보통이에요/3
최고예요/5
no review in crawling
no review in crawling
#### 카페소재
입구 바 테이블에 앉으면 공부하기 좋다.  커피 맛도 ㄱㅊ /5
우선 음료랑 크로플 다 너무 맛있어요. 사장님과 직원분 진짜 친절하시고, 먹다가 메이플시럽하고 시나몬 가루 부탁드렸는데 추가로 더 주시고 감사했습니다. 에이드도 진짜 상큼하고 비주얼 대박이에요. 무엇보다 방역 철저하게 하시는 거 같아 좋았어요. 다른 손님 나가자마자 테이블이랑 쟁반 다 소독하시는거 봤어요. 집근처였으면 진짜 매일 가고 싶네요. 친구랑 왕십리.../5
크로플 존맛탱 ㅜㅠ/5
즉석에서 만들어주는 크로플도 맛있고, 로얄 밀크티도 내 취향이다. 매장도 널찍하고 집 근처였으면 자주 들렀을 거 같음/3
최고예요/5
크로플 너무너무 맛있고ㅠㅠ 음료도 맛있고! 공간도 넓고 쾌적하고 일하시는분들도 친절하시다/5
인테리어 널찍하고 무엇보다도 커피랑 디저트 다 맛있어요/5
인테리어 예뻐요/5
no review in crawling
no review in crawling
#### 이디야커피 왕십리비트플렉스점
불친절함. QR결제 안된다고 함./2
no review in crawling
no review in crawling
#### 할리스커피 성동구청점
최고예요/5
가보시면 압니다../1
보통이에요/3
보통이에요/3
먹으며 얄짤없이 1킬로 찌는 할리스 허니 바게트볼 ㅋ/0
요기 스프 아주맛나요~ 빵은 쨈에 찍어먹고 스프랑같이 긁어먹고 한끼식사대용~/0
no review in crawling
no review in crawling
#### 통툰 왕십리역점
방마다 cctv가 있어서 좀;;/2
오지마세요 보는 눈이 많아요/1
진짜 1점도 아까울 정도로 사장의 불친절한 손님대응이 기분을 안좋게 만드네요. 나만 그런 줄 알았는데 이미 많은 분

WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.182)


In [ ]:
searchList

In [45]:
df = pd.DataFrame(searchList)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1361 non-null   object
 1   1       1361 non-null   object
 2   2       1361 non-null   object
dtypes: object(3)
memory usage: 32.0+ KB


In [47]:
df.head(10)

,0,1,2
0,맥심플랜트,분위기도 좋고 커피도 맛있고 공간이 머쉿다~,4점
1,맥심플랜트,최고예요,5점
2,맥심플랜트,좋아요,4점
3,맥심플랜트,공부하기 좋고 넓어서 애용하는 곳,5점
4,맥심플랜트,좋아요,4점
5,맥심플랜트,이가격에 이공간 이맛 끄덕끄덕,5점
6,맥심플랜트,최고예요,5점
7,맥심플랜트,음료보다 인테리어가 더 인상깊어요 넓고 공간도 다양하게 있습니다 꼭대기층 테라스자리...,4점
8,맥심플랜트,요 근처 회사 다니는 사람이에요. 커피 맛은 대단히 맛있지도 별로지도 아닌 보통의 ...,3점
9,맥심플랜트,커피맛 최고! 아메리카노 골든스카이 + 글래머러스 펭귄 케익 최고!! 공간 다양 ...,4점


In [48]:
df.to_csv("왕십리 cafe_review.csv")

In [19]:
cafe_namelist = cafe_list.drop_duplicates()
cafe_namelist

NameError: name 'cafe_list' is not defined

In [ ]:
cafe_namelist = pd.DataFrame(cafe_namelist)
cafe_namelist

In [ ]:
cafe_namelist['cafe_name'] = "강남카페 " + cafe_namelist[0]

In [ ]:
cafe_namelist

In [ ]:
cafe_namelist.drop([0], axis=1)

In [ ]:
cafe_namelist.to_csv("cafe_namelist.csv")